# Project 009 - Transfer Learning with RESNET50 -  Keras Implementation


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#(You can swap this out for your own dataset later.)
#Save CIFAR-10 images to folders

import os
import cv2
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def save_split(split_name, images, labels):
    base = f"cifar10_{split_name}"
    os.makedirs(base, exist_ok=True)

    for i, (img, lbl) in enumerate(zip(images, labels)):
        cls = classes[lbl[0]]
        cls_path = os.path.join(base, cls)
        os.makedirs(cls_path, exist_ok=True)
        cv2.imwrite(os.path.join(cls_path, f"{i}.png"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

save_split("train", x_train, y_train)
save_split("test", x_test, y_test)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
#Use a generator that automatically resizes to 224×224

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    "cifar10_train",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "cifar10_test",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:
#Apply ResNet50 Preprocessing

x_train = resnet50.preprocess_input(x_train)
x_test = resnet50.preprocess_input(x_test)

In [5]:
#Load Pre-trained ResNet50 (without top layer)

base_model = ResNet50(
    weights="imagenet",
    include_top=False, #this is to help us change the 1000 classes from ImageNet for us to add our own
    input_shape=(224, 224, 3) #this defines the expected input to be feed into the ResNet model
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
#Add Custom Classification Head

num_classes = 10   # <---- ADD THIS

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [7]:
#Compile the Model

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [12]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

In [8]:
#Train the Classification Head Only (Feature Extraction)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=7
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 183s 108ms/step - accuracy: 0.6227 - loss: 1.1225 - val_accuracy: 0.7709 - val_loss: 0.6419
Epoch 2/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 159s 102ms/step - accuracy: 0.7453 - loss: 0.7338 - val_accuracy: 0.8023 - val_loss: 0.5676
Epoch 3/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 159s 102ms/step - accuracy: 0.7726 - loss: 0.6600 - val_accuracy: 0.8091 - val_loss: 0.5422
Epoch 4/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 158s 101ms/step - accuracy: 0.7779 - loss: 0.6278 - val_accuracy: 0.8088 - val_loss: 0.5524
Epoch 5/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 159s 102ms/step - accuracy: 0.7957 - loss: 0.5931 - val_accuracy: 0.8167 - val_loss: 0.5243
Epoch 6/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 159s 102ms/step - accuracy: 0.7977 - loss: 0.5860 - val_accuracy: 0.8179 - val_loss: 0.5214
Epoch 7/7
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 159s 102ms/step - accuracy: 0.8083 - loss: 0.5499 - val_accuracy: 0.8245 - val_loss: 0.5105


In [9]:
#Fine-Tune the Deeper ResNet Layers (Optional but increases accuracy)

for layer in base_model.layers[-30:]:   # Unfreeze last 30 layers
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 243s 143ms/step - accuracy: 0.7717 - loss: 0.7055 - val_accuracy: 0.8623 - val_loss: 0.4125
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 213s 136ms/step - accuracy: 0.8749 - loss: 0.3650 - val_accuracy: 0.8788 - val_loss: 0.3715
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 214s 137ms/step - accuracy: 0.9160 - loss: 0.2437 - val_accuracy: 0.8890 - val_loss: 0.3689
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 213s 137ms/step - accuracy: 0.9436 - loss: 0.1682 - val_accuracy: 0.8860 - val_loss: 0.3825
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 214s 137ms/step - accuracy: 0.9618 - loss: 0.1172 - val_accuracy: 0.8899 - val_loss: 0.3996


In [13]:
#Evaluate

loss, acc = model.evaluate(x_test_resized, y_test)
print("Final Test Accuracy:", acc)

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 10)

#Predict on an External Image

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_external(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet50.preprocess_input(img_array)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    print("Predicted Class:", predicted_class)

    plt.imshow(image.load_img(img_path))
    plt.title("Predicted: " + str(predicted_class))
    plt.axis("off")
    plt.show()

predict_external("014.png")
